In [1]:
# Import libraries
import pandas as pd
from math import sqrt
import numpy as np

In [2]:

import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bf63d8034a7a4cfe81617b85e5075c04 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='8vf1KdAjYDuS5Josd_IH_vmehb7EfGU7eRBglM1Zgwlq',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bf63d8034a7a4cfe81617b85e5075c04.get_object(Bucket='movierecommender-donotdelete-pr-8pww64483vjeew',Key='movies.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1
movies_df = df_data_1
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# User's ratings
body = client_bf63d8034a7a4cfe81617b85e5075c04.get_object(Bucket='movierecommender-donotdelete-pr-8pww64483vjeew',Key='userRatings.csv')['Body'] 
#add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_3 = pd.read_csv(body)
user_ratings_df = df_data_3
# Change to title and rating to match columns in movies_df
user_ratings_df.rename(columns={'Title':'title', 'Rating':'rating'}, inplace=True)
user_ratings_df.head()

,title,rating
0,"Matrix, The",5.0
1,Risen,5.0
2,Toy Story,4.5
3,"Breakfast Club, The",4.5
4,Jumanji,1.0


In [5]:
# Create new year column, remove year from title column
movies_df['year'] = movies_df['title']
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji (1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II (1995)


In [6]:
# Get rid of year from title column
movies_df['title'] = movies_df['title'].replace(regex=r'\(\d\d\d\d\)', value='')
# Get rid of any trailing white spaces in title
movies_df['title'] = movies_df['title'].str.strip()
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,Toy Story (1995)
1,2,Jumanji,Adventure|Children|Fantasy,Jumanji (1995)
2,3,Grumpier Old Men,Comedy|Romance,Grumpier Old Men (1995)
3,4,Waiting to Exhale,Comedy|Drama|Romance,Waiting to Exhale (1995)
4,5,Father of the Bride Part II,Comedy,Father of the Bride Part II (1995)


In [7]:
movies_df['year'] = movies_df['year'].replace(regex=r'.*\(', value='')
movies_df['year'] = movies_df['year'].replace(regex=r'\)$', value='')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [8]:
# Split genres into list format
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [9]:
# Create a table with genres set to 1
movies_genres_df = movies_df.copy()
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        movies_genres_df.at[index, genre] = 1
movies_genres_df = movies_genres_df.fillna(0)
movies_genres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Get movieId and add it to user_ratings_df
user_ratings_merged_df = user_ratings_df.merge(movies_genres_df, left_on='title', 
                                            right_on='title', how='inner')
user_ratings_merged_df

,title,rating,movieId,genres,year,Adventure,Animation,Children,Comedy,Fantasy,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,"Matrix, The",5.0,2571,"[Action, Sci-Fi, Thriller]",1999,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Risen,5.0,151687,"[Children, Drama]",2016,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Toy Story,4.5,1,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Breakfast Club, The",4.5,1968,"[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Jumanji,1.0,2,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Akira,0.0,1274,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Pulp Fiction,2.5,296,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Bloodmoney,2.0,151701,[(no genres listed)],2010,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,Speed,5.0,377,"[Action, Romance, Thriller]",1994,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Scream,5.0,1407,"[Comedy, Horror, Mystery, Thriller]",1996,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Drop genres, year and reset_index
user_ratings_merged_df.drop(['genres', 'year'], axis=1, inplace=True)
user_ratings_merged_df

,title,rating,movieId,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,"Matrix, The",5.0,2571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Risen,5.0,151687,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Toy Story,4.5,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Breakfast Club, The",4.5,1968,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Jumanji,1.0,2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Akira,0.0,1274,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Pulp Fiction,2.5,296,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Bloodmoney,2.0,151701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,Speed,5.0,377,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Scream,5.0,1407,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Take dot product of genres tables and user rating
user_ratings_profile = user_ratings_merged_df.iloc[:, 3:].transpose().dot(user_ratings_merged_df['rating'])
user_ratings_profile

Adventure              5.5
Animation              4.5
Children              10.5
Comedy                16.5
Fantasy                5.5
Romance                5.0
Drama                 12.0
Action                10.0
Crime                  2.5
Thriller              17.5
Horror                 5.0
Mystery                5.0
Sci-Fi                 5.0
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     2.0
dtype: float64

In [13]:
# Calculate weight of genres based on user ratings
weighted_user_ratings = user_ratings_profile/user_ratings_profile.sum()
weighted_user_ratings

Adventure             0.051643
Animation             0.042254
Children              0.098592
Comedy                0.154930
Fantasy               0.051643
Romance               0.046948
Drama                 0.112676
Action                0.093897
Crime                 0.023474
Thriller              0.164319
Horror                0.046948
Mystery               0.046948
Sci-Fi                0.046948
IMAX                  0.000000
Documentary           0.000000
War                   0.000000
Musical               0.000000
Western               0.000000
Film-Noir             0.000000
(no genres listed)    0.018779
dtype: float64

In [14]:
# Multiply user profile matrix with movies matrix
genre_table = movies_genres_df.set_index(movies_genres_df['movieId'])
genre_table = genre_table.drop(['movieId', 'title', 'genres', 'year'], axis=1)
genre_table.head()
weighted_matrix = weighted_user_ratings*genre_table
weighted_matrix.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,0.051643,0.042254,0.098592,0.15493,0.051643,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.051643,0.000000,0.098592,0.00000,0.051643,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.15493,0.000000,0.046948,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.15493,0.000000,0.046948,0.112676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.000000,0.15493,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Weighted average matrix
recommend_movies = weighted_matrix.sum(axis=1)
recommend_movies.head()

movieId
1    0.399061
2    0.201878
3    0.201878
4    0.314554
5    0.154930
dtype: float64

In [16]:
# Sort recommendation matrix in descending order to find highest recommendations
highest_recommended_movies = recommend_movies.sort_values(ascending=False).head(20)
highest_recommended_movies

movieId
81132     0.694836
75408     0.685446
76153     0.685446
149488    0.676056
83266     0.671362
5018      0.652582
4719      0.638498
6902      0.629108
117646    0.629108
4956      0.624413
139159    0.624413
64645     0.600939
122787    0.600939
7235      0.596244
27781     0.596244
127341    0.596244
55116     0.577465
27735     0.577465
58718     0.577465
51545     0.572770
dtype: float64

In [24]:
# Final movie recommendations
final_recommendations_df = movies_df.loc[movies_df['movieId'].isin(highest_recommended_movies.keys())]
final_recommendations_df.reset_index(inplace=True, drop=True)
final_recommendations_df

,movieId,title,genres,year
0,4719,Osmosis Jones,"[Action, Animation, Comedy, Crime, Drama, Roma...",2001
1,4956,"Stunt Man, The","[Action, Adventure, Comedy, Drama, Romance, Th...",1980
2,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
3,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
4,7235,Ichi the Killer (Koroshiya 1),"[Action, Comedy, Crime, Drama, Horror, Thriller]",2001
5,27735,Unstoppable,"[Action, Adventure, Comedy, Drama, Thriller]",2004
6,27781,Svidd Neger,"[Comedy, Crime, Drama, Horror, Mystery, Romanc...",2003
7,51545,Pusher III: I'm the Angel of Death,"[Action, Comedy, Drama, Horror, Thriller]",2005
8,55116,"Hunting Party, The","[Action, Adventure, Comedy, Drama, Thriller]",2007
9,58718,Slipstream,"[Comedy, Drama, Fantasy, Mystery, Sci-Fi, Thri...",2007
